## Configuration and Intercomparison of Deep Learning Neural Models for Statistical Downscaling *(code for full reproducibility)*
### *Geoscientific Model Development preprint*
### J. Baño-Medina, R. Manzanas and J. M. Gutiérrez
https://doi.org/10.5194/gmd-2019-278

GitHub repository at https://github.com/SantanderMetGroup/DeepDownscaling

This notebook reproduces the results presented in the paper *Configuration and Intercomparison of Deep Learning Neural Models for Statistical Downscaling* by *J. Baño-Medina, R. Manzanas and J. M. Gutiérrez*, in *Geoscientific Model Development* (https://doi.org/10.5194/gmd-2019-278). In particular, the code developed herein concerns the downscaling of temperature and precipitation and therefore, their particularities are treated throughout the notebook in two different sections. Note that the programming lenguage is `R` and the technical specifications of the machine can be found at the end of the notebook. **The notebook takes around 5 days to fully reproduce the results. For this reason we refer the reader to a reduced version of the notebook in which we only consider a particular model configuration ([2020_Bano_GMD.ipynb](https://github.com/SantanderMetGroup/DeepDownscaling))**.

**Note:** This notebook is written in the free programming language `R` and builds on the `R` framework [`climate4R`](https://github.com/SantanderMetGroup/climate4R) (C4R hereafter, conda and docker installations available), a suite of `R` packages developed by the [Santander Met Group](http://meteo.unican.es) for transparent climate data access, post processing (including bias correction and downscaling, via the [`downscaleR`](https://github.com/SantanderMetGroup/downscaleR) package; [Bedia et al. 2020](https://www.geosci-model-dev-discuss.net/gmd-2019-224/)) and visualization. The interested reader is referred to [Iturbide et al. 2019](https://www.sciencedirect.com/science/article/pii/S1364815218303049?via%3Dihub). 

## 1. Loading libraries


All the working steps rely on the [climate4R](https://github.com/SantanderMetGroup/climate4R) (conda and docker installations available), which is a set of libraries especifically developed to handle climate data (`loadeR`,`transformeR`,`downscaleR`, `visualizeR` and `climate4R.value`). In this study, [C4R](https://github.com/SantanderMetGroup/climate4R) is used for loading and post-processing, downscaling, validation and visualization. Different sectorial [notebooks](https://github.com/SantanderMetGroup/notebooks) are available illustrating the use of C4R functions. 

Deep learning models are indcluded as an extension of the downscaleR package: [`downscaleR.keras`](https://github.com/SantanderMetGroup/downscaleR.keras) which integrates *keras* in the C4R framework.

Here we use a **specific version of the conda C4R installer (v1.3.0)** containing the package versions needed to fully reproduce the results of this notebook and including also the [`downscaleR.keras`](https://github.com/SantanderMetGroup/downscaleR.keras) package which is not included in the standard C4R conda distribution (visit this [site](https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html) for more information on how to obtain conda). Thus, below we illustrate how to create a new environment with conda to install the C4R libraries (please visit the [`climate4R`](https://github.com/SantanderMetGroup/climate4R) github repository for more details regarding the installation procedures). Note that the packages can be also installed individually, but this is more time consuming than using the conda installation.

In [ ]:
## --- type this in terminal to proceed with the installation of C4R with conda

# conda create --name myEnvironment  # create a new environment
# conda activate myEnvironment  # activate the environment
# conda install -c conda-forge -c defaults -c santandermetgroup climate4r=1.3.0  # install climate4R v1.3.0 in the new environment
# R # type 'R' to initialize R software

Once in R, we load the libraries:

In [2]:
# C4R framework and auxiliary packages

library(loadeR) # version 1.6.1      
library(transformeR) # version 1.7.4
library(downscaleR) # version 3.1.3
library(visualizeR) # version 1.5.1
library(climate4R.value) # version 0.0.2 (also relies on VALUE version 2.2.1)
library(magrittr)
library(gridExtra)
library(RColorBrewer)
library(sp)            
library(downscaleR.keras) # version 0.0.2 (relies on keras version 2.2.2 and tensorflow version 2.0.0)

In order to avoid possible errors while running the notebook, you have to set the path to your desired working directory and create two files named "Data" and "models", that will contain the downscaled predictions and the trained deep models, respectively. Moreover, as we perform 2 distinct studies, one for precipitation and other for temperature, you should create 2 new directories named "precip" and "temperature" within the previous created directories (i.e., "Data" and "models"). An example of the latter would be "personalpath/Data/temperature". The predictions and models infered will be automatically saved in these folders and therefore not creating them will end into saving errors across the notebook.

In [ ]:
path = ""
setwd(path)
dir.create("Data")
dir.create("Data/precip/")
dir.create("models")
dir.create("models/temperature/")
dir.create("models/precip/")

## 2. Loading data

As explained in the paper, we have considered 20 large-scale variables from the ERA-Interim reanalysis as predictors and surface temperature from E-OBS as predictand. All these data can be loaded remotely from the [Santander Climate Data Service](http://meteo.unican.es/cds) (register [here](https://meteo.unican.es/trac/wiki/udg) freely to get a user), which provides access to various kinds of climate datasets (global and regional climate models, reanalysis, observations...). We will use the C4R packages [`loadeR`](https://github.com/SantanderMetGroup/loadeR) and [`transformeR`](https://github.com/SantanderMetGroup/transformeR) to load and postprocess the required information.

In [ ]:
loginUDG(username = "", password = "") # login into the Santander CDS

We find the label associated to ERA-Interim via the `UDG.datasets()` function of loadeR: “ECMWF_ERA-Interim-ESD”. Then we load the predictors by calling `loadGridData` from loadeR.

In [ ]:
variables <- c("z@500","z@700","z@850","z@1000",
               "hus@500","hus@700","hus@850","hus@1000",
               "ta@500","ta@700","ta@850","ta@1000",
               "ua@500","ua@700","ua@850","ua@1000",
               "va@500","va@700","va@850","va@1000")
x <- lapply(variables, function(x) {
  loadGridData(dataset = "ECMWF_ERA-Interim-ESD",
               var = x,
               lonLim = c(-10,32), # 22 points en total
               latLim = c(36,72),  # 19 points en total
               years = 1979:2008)
}) %>% makeMultiGrid()

We split into train (i.e., 1979-2002) and test (i.e., 2003-2008) datasets by using the `subsetGrid` function from transformeR.

In [ ]:
xT <- subsetGrid(x,years = 1979:2002)
xt <- subsetGrid(x,years = 2003:2008)

## 3. Temperature
In this section we present the code needed to downscale temperature.  Once the predictors were loaded above we proceed to download the predictand dataset: E-OBS version 14 at a resolution of 0.5º. The E-OBS dataset is also accesible through the UDG portal. Thus, we load the temperature by calling again `loadGridData` and split it into train and test periods.

In [ ]:
y <- loadGridData(dataset = "E-OBS_v14_0.50regular",
                  var = "tas",
                  lonLim = c(-10,32),
                  latLim = c(36,72), 
                  years = 1979:2008)

yT <- subsetGrid(y,years = 1979:2002)
yt <- subsetGrid(y,years = 2003:2008)

We can take a look at the grid's resolutions of ERA-Interim (2º) and E-OBS (0.5º) in order to better visualize the gap we try to bridge with the downscaling (Figure 1 of the manuscript).

In [ ]:
cb <- colorRampPalette(brewer.pal(9, "OrRd"))(80)
coords_x <- expand.grid(xt$xyCoords$x,xt$xyCoords$y) ; names(coords_x) <- c("x","y")
coords_y <- expand.grid(yt$xyCoords$x,yt$xyCoords$y) ; names(coords_y) <- c("x","y")
colsindex <- rev(brewer.pal(n = 9, "RdBu"))
cb2 <- colorRampPalette(colsindex)

pplot <- list()
pplot[[1]] <- spatialPlot(climatology(subsetGrid(xt,var = "ta@1000")), backdrop.theme = "coastline",
                          main = "Temperature 1000 hPa (ERA-Interim)",
                          col.regions = cb2,
                          at = seq(-3, 15, 1),
                          set.min = -3, set.max = 15, colorkey = TRUE,
                          sp.layout = list(list(SpatialPoints(coords_x), 
                                                first = FALSE, col = "black", 
                                                pch = 20, cex = 1)))
pplot[[2]] <- spatialPlot(climatology(yt), backdrop.theme = "coastline", 
                          main = "Temperature (E-OBS)",
                          col.regions = cb,
                          at = seq(-3, 15, 1),
                          set.min = -3, set.max = 15, colorkey = TRUE,
                          sp.layout = list(list(SpatialPoints(coords_y), 
                                                first = FALSE, col = "black", 
                                                pch = 20, cex = 1)))

lay = rbind(c(1,2))
grid.arrange(grobs = pplot, layout_matrix = lay)

We can visualize some statistics of the train and test distributions, such as the climatology, or the percentiles 02th and 98th in order to gain knowledge about the observed data (Figure 2 of the manuscript). To compute the statistics we use `valueIndex` and `valueMeasure` functions from `climate4R.value` , a wrapper to the [`VALUE`](https://github.com/SantanderMetGroup/VALUE) package, which was developed in the [COST action VALUE](http://www.value-cost.eu/) for validation purposes. If interested in knowing more about the plotting options used here, the reader is referred to the documentation of the `spatialPlot` function.

In [ ]:
cb <- colorRampPalette(brewer.pal(9, "OrRd"))(80)
colsindex <- rev(brewer.pal(n = 9, "RdBu"))
cb2 <- colorRampPalette(colsindex)

pplot <- at <- list()
n1 <- 0; n2 <- 3
indexNames <- c("Climatology", "P02", "P98")
for (indexName in indexNames) {
  if (indexName == "Climatology") {
    indexTrain <- valueIndex(yT,index.code = "Mean")$Index %>% redim()  
    indexTest <- valueIndex(yt,index.code = "Mean")$Index %>% redim()
    at[[1]] <- seq(-3, 15, 1); at[[2]] <- seq(-2, 2, 0.1)
  }
  if (indexName == "P02") {
    indexTrain <- valueIndex(yT,index.code = "P02")$Index %>% redim()  
    indexTest <- valueIndex(yt,index.code = "P02")$Index %>% redim()
    at[[1]] <- seq(-20, 10, 1); at[[2]] <- seq(-2, 2, 0.1)
  }
  if (indexName == "P98") {
    indexTrain <- valueIndex(yT,index.code = "P98")$Index %>% redim()  
    indexTest <- valueIndex(yt,index.code = "P98")$Index %>% redim()
    at[[1]] <- seq(10, 30, 1); at[[2]] <- seq(-2, 2, 0.1)
  }
  
  for (i in 1:2) {
    if (i == 1) {
      dataset <- "(train)"; index <- indexTrain; n1 <- n1 + 1; n <- n1
      value <- index$Data; colorbar <- cb
    }
    if (i == 2) {
      indexTest <- gridArithmetics(indexTest,indexTrain,operator = "-")
      dataset <- "(test bias)"; index <- indexTest; n2 <- n2 + 1; n <- n2
      value <- abs(index$Data); colorbar <- cb2
    }
    pplot[[n]] <- spatialPlot(climatology(index), backdrop.theme = "coastline", 
                              main = paste(indexName,paste0(dataset,":"),
                                           round(mean(value, na.rm = TRUE),digits = 2)),
                              col.regions = colorbar,
                              at = at[[i]],
                              set.min = at[[i]][1], set.max = at[[i]][length(at[[i]])], 
                              colorkey = TRUE)
  }
}

lay = rbind(c(1,2,3),
            c(4,5,6))
grid.arrange(grobs = pplot, layout_matrix = lay)

Once the data is loaded we standardize the predictors by calling `scaleGrid` function of transformeR.

In [ ]:
xt <- scaleGrid(xt,xT, type = "standardize", spatial.frame = "gridbox") %>% redim(drop = TRUE)
xT <- scaleGrid(xT, type = "standardize", spatial.frame = "gridbox") %>% redim(drop = TRUE)

### 3.1 Downscaling with generalized linear models (temperature)

To downscale via generalized linear models (GLM) we rely on the `downscaleR` package of `C4R`. In particular, we use the `downscaleChunk` function of `downscaleR`. In the case of temperature, the generalized linear model has a gaussian family with link identity which is, in fact, an ordinary least squares regression. Therefore, we input to the function the predictor (x), the number of local predictors to be used (neighbours), the predictand (y) and the test set to apply the infered relationship as newdata. Finally we save the predictions which will be loaded during validation. Note that `downscaleChunk` temporarily creates .rda files in your working directory, containing the predictions per chunk. 

In [ ]:
glmName <- c("glm1","glm4")
neighs <- c(1,4)
lapply(1:length(glmName), FUN = function(z) {
  pred <- downscaleChunk(x = xT, y = yT, newdata = list(xt),
                         method = "GLM", family = "gaussian", simulate = FALSE,
                         prepareData.args = list(local.predictors = list(n=neighs[z], 
                                                                         vars = getVarNames(xT))))[[2]] %>% 
    redim(drop = TRUE)
  save(pred,file = paste0("./Data/temperature/predictions_",glmName[z],".rda"))
})

### 3.2 Downscaling with deep neural networks

In the following code we define a function containing the deep learning topologies intercompared in the study (see Table 2 and Figure 3 of the manuscript for more information about these architectures).

In [ ]:
deepName <- c("CNN-LM","CNN1","CNN10","CNN-PR","CNNdense")
architectures <- function(architecture,input_shape,output_shape) {
  if (architecture == "CNN-LM") {
    inputs <- layer_input(shape = input_shape)
    x = inputs
    l1 = layer_conv_2d(x ,filters = 50, kernel_size = c(3,3), activation = 'linear', padding = "same")
    l2 = layer_conv_2d(l1,filters = 25, kernel_size = c(3,3), activation = 'linear', padding = "same")
    l3 = layer_conv_2d(l2,filters = 1, kernel_size = c(3,3), activation = 'linear', padding = "same")
    l4 = layer_flatten(l3)
    outputs = layer_dense(l4,units = output_shape)
    model <- keras_model(inputs = inputs, outputs = outputs)
  }
  
  if (architecture == "CNN1") {
    inputs <- layer_input(shape = input_shape)
    x = inputs
    l1 = layer_conv_2d(x ,filters = 50, kernel_size = c(3,3), activation = 'relu', padding = "same")
    l2 = layer_conv_2d(l1,filters = 25, kernel_size = c(3,3), activation = 'relu', padding = "same")
    l3 = layer_conv_2d(l2,filters = 1, kernel_size = c(3,3), activation = 'relu', padding = "same")
    l4 = layer_flatten(l3)
    outputs = layer_dense(l4,units = output_shape)
    model <- keras_model(inputs = inputs, outputs = outputs)
  }
  
  if (architecture == "CNN10") {
    inputs <- layer_input(shape = input_shape)
    x = inputs
    l1 = layer_conv_2d(x ,filters = 50, kernel_size = c(3,3), activation = 'relu', padding = "valid")
    l2 = layer_conv_2d(l1,filters = 25, kernel_size = c(3,3), activation = 'relu', padding = "valid")
    l3 = layer_conv_2d(l2,filters = 10, kernel_size = c(3,3), activation = 'relu', padding = "valid")
    l4 = layer_flatten(l3)
    outputs = layer_dense(l4,units = output_shape)
    model <- keras_model(inputs = inputs, outputs = outputs)
  }
  
  if (architecture == "CNN-PR") {
    inputs <- layer_input(shape = input_shape)
    x = inputs
    l1 = layer_conv_2d(x ,filters = 10, kernel_size = c(3,3), activation = 'relu', padding = "valid")
    l2 = layer_conv_2d(l1,filters = 25, kernel_size = c(3,3), activation = 'relu', padding = "valid")
    l3 = layer_conv_2d(l2,filters = 50, kernel_size = c(3,3), activation = 'relu', padding = "valid")
    l4 = layer_flatten(l3)
    outputs = layer_dense(l4,units = output_shape)
    model <- keras_model(inputs = inputs, outputs = outputs)
  }
  
  if (architecture == "CNNdense") {
    inputs <- layer_input(shape = input_shape)
    x = inputs
    l1 = layer_conv_2d(x ,filters = 50, kernel_size = c(3,3), activation = 'relu', padding = "valid")
    l2 = layer_conv_2d(l1,filters = 25, kernel_size = c(3,3), activation = 'relu', padding = "valid")
    l3 = layer_conv_2d(l2,filters = 10, kernel_size = c(3,3), activation = 'relu', padding = "valid")
    l4 = layer_flatten(l3)
    l5 = layer_dense(l4,units = 50, activation = "relu")
    l6 = layer_dense(l5,units = 50, activation = "relu")
    outputs = layer_dense(l6,units = output_shape)
    model <- keras_model(inputs = inputs, outputs = outputs)
  }
  
  return(model)
}

We prepare the predictor and predictand datasets for integration with keras with the functions `prepareData.keras` and `prepareNewData.keras`

In [ ]:
xy.T <- prepareData.keras(xT,yT,
                          first.connection = "conv",
                          last.connection = "dense",
                          channels = "last")
xy.t <- prepareNewData.keras(xt,xy.T)

We loop over the topologies to train the deep models and predict over the test set. Unlike GLMs where there was a model per gridpoint, deep models perform multi-task, downscaling to all sites at a time.

In [ ]:
lapply(1:length(deepName), FUN = function(z){
  model <- architectures(architecture = deepName[z],
                         input_shape = dim(xy.T$x.global)[-1],
                         output_shape = dim(xy.T$y$Data)[2])
  downscaleTrain.keras(obj = xy.T,
                       model = model,
                       clear.session = TRUE,
                       compile.args = list("loss" = "mse",
                                           "optimizer" = optimizer_adam(lr = 0.0001)),
                       fit.args = list("batch_size" = 100,
                                       "epochs" = 1000,
                                       "validation_split" = 0.1,
                                       "verbose" = 1,
                                       "callbacks" = list(callback_early_stopping(patience = 30),
                                                 callback_model_checkpoint(
                                                 filepath=paste0('./models/temperature/',deepName[z],'.h5'),
                                                                  monitor='val_loss', save_best_only=TRUE))))
  pred <- downscalePredict.keras(newdata = xy.t,
                                 model = list("filepath" = 
                                                paste0("./models/temperature/",deepName[z],".h5")),
                                 C4R.template = yT,
                                 clear.session = TRUE)
  save(pred,file = paste0("./Data/temperature/predictions_",deepName[z],".rda"))
})

### 3.3 Validation of results
In this code, we calculate the validation indices by using the library `climate4R.value` of `C4R`. In particular the indices used are: the Root Mean Squared Error (RMSE), the deseasonal perason correlation, the biases of the climatology and of the percentile 2th and 98th and the ratio of the standard deviations.

In [ ]:
models <- c("glm1","glm4",
            "CNN-LM","CNN1","CNN10",
            "CNN-PR","CNNdense")
measures <- c("ts.RMSE","ts.rp","ratio",rep("bias",6))
index <- c(rep(NA,2),"sd","Mean","P02","P98","AC1",
           "WarmAnnualMaxSpell","ColdAnnualMaxSpell")
yt2 <- scaleGrid(yt,time.frame = "daily",window.width = 31) %>% redim(drop=TRUE)
validation.list <- lapply(1:length(measures), FUN = function(z) {
  lapply(1:length(models), FUN = function(zz){
    args <- list()
    load(paste0("./Data/temperature/predictions_",models[zz],".rda"))
    if (any(measures[z] == c("ts.rp","ratio"))) {
      pred2 <- scaleGrid(pred,time.frame = "daily",window.width = 31) %>% redim(drop=TRUE)
      args[["y"]] <- yt2; args[["x"]] <- pred2
    } else {
      args[["y"]] <- yt; args[["x"]] <- pred
    }
    args[["measure.code"]] <- measures[z]
    if (!is.na(index[z])) args[["index.code"]] <- index[z]
    do.call("valueMeasure",args)$Measure
  }) %>% makeMultiGrid()
})
save(validation.list, file = "./Data/temperature/validation.rda")

Once the validation indices are calculated, we represent the results in boxplots (Figure 4 of the manuscript).

In [ ]:
ylabs <- c("RMSE (ºC)","Cor. deseasonal",
           "Standard dev. ratio","bias (ºC)",
           "bias P02 (ºC)","bias P98 (ºC)",
           "bias AC1", "bias WAMS (days)",
           "bias CAMS (days)")
par(mfrow = c(3,3))
lapply(1:length(validation.list), FUN = function(z) {
  if (z == 1) {ylim <- c(0.75,1.75)}
  if (z == 2) {ylim <- c(0.9,1)}
  if (z == 3) {ylim <- c(0.9,1.05)}
  if (z == 4) {ylim <- c(-0.2,0.5)}
  if (z == 5) {ylim <- c(-0.5,1.5)}
  if (z == 6) {ylim <- c(-1,1)}
  if (z == 7) {ylim <- c(-0.1,0.1)}
  if (any(z == c(8,9))) {ylim <- c(-3,3)}
  index <- (validation.list[[z]] %>% redim(drop = TRUE))$Data
  dim(index) <- c(nrow(index),prod(dim(index)[2:3]))
  indLand <- (!apply(index,MARGIN = 2,anyNA)) %>% which()
  index <- index[,indLand] %>% t()
  mglm4 <- median(index[,2],na.rm = TRUE)
  perc <- apply(index,MARGIN = 2,FUN = function(z) quantile(z,probs = c(0.1,0.9)))
  boxplot(index, outline = FALSE, ylim = ylim, range = 0.0001, ylab = ylabs[z], asp = 1)
  lines(c(0,8),c(mglm4,mglm4), col = "red")
  for (i in 1:ncol(index)) lines(c(i,i),perc[,i], lty = 2)
})

In order to obtain a spatial representation of the validation indices computed above we use the function `spatialPlot` of visualizeR. In particular, we plot the deseasonal correlation and the biases of the percentile 2th, 98th and the mean for the glm1, glm4 and CNN10 models (Figure 5 of the manuscript).

In [ ]:
ylabs <- c("glm1","glm4",NA,NA,"CNN10")
mains <- c(NA,"Cor. deseasonal",NA,"bias (ºC)","bias P02 (ºC)","bias P98 (ºC)")
cb <- colorRampPalette(brewer.pal(9, "OrRd"))(80)
colsindex <- rev(brewer.pal(n = 9, "RdBu"))
cb2 <- colorRampPalette(colsindex)
validation.plots <- lapply(c(2,4,5,6),FUN = function(z) {
  lapply(c(1,2,5),FUN = function(zz) {
    if (z == 2) {
      at <- seq(0.85, 1, 0.005); colorbar <- cb
    } else {
      at <- seq(-2, 2, 0.1); colorbar <- cb2
    }
    index <- subsetDimension(validation.list[[z]],dimension = "var",indices = zz) %>% 
      redim(drop = TRUE)
    spatialPlot(index, backdrop.theme = "coastline",
                ylab = ylabs[zz],
                main = paste(mains[z],
                             round(mean(abs(index$Data), na.rm = TRUE), digits = 2)),
                col.regions = colorbar,
                at = at,
                set.min = at[1], set.max = at[length(at)], colorkey = TRUE)
  })
})
lay = cbind(1:3,4:6,7:9,10:12)
grid.arrange(grobs = unlist(validation.plots,recursive = FALSE), layout_matrix = lay)

To provide more insight into the extrapolation abilities of the statistical downscaling methods tested herein, we compute the bias of the frequency of days above the percentile 95th (over the train period) in the observed and predicted (GLM4 and CNN10) time series with respect to the same frequency in the observed train values.

In [ ]:
grids <- c("yt","predictions_glm1","predictions_glm4","predictions_CNN10")
gridsNames <- c("Test","GLM1","GLM4","CNN10")
p.plots <- lapply(c(0.95,0.99), FUN = function(zzz){
  pX <- apply(yT$Data,MARGIN = c(2,3),FUN = function(z) quantile(z,probs = zzz,na.rm=TRUE))
  if (zzz == 0.95) at <- seq(-0.1, 0.1, length.out = 30)
  if (zzz == 0.99) at <- seq(-0.05, 0.05, length.out = 30)
  lapply(1:length(grids),FUN = function(zz) {
    grid <- get(load(paste0("./Data/temperature/",grids[zz],".rda")))
    freq.t <- grid
    dimNames <- attr(grid$Data,"dimensions")
    days.t <- lapply(1:getShape(grid,"time"),FUN = function(z){
      grid$Data[z,,] > pX 
    }) %>% abind::abind(along = 0)
    freq.t$Data <- apply(days.t,MARGIN = c(2,3),FUN = function(z) mean(z,na.rm = TRUE))
    attr(freq.t$Data,"dimensions") <- dimNames[-1]
    freq.t <- gridArithmetics(freq.t,1-zzz,operator = "-")
    spatialPlot(freq.t,
                backdrop.theme = "coastline",
                col.regions = rev(brewer.pal(n = 9, "RdBu")) %>% colorRampPalette(),
                main = gridsNames[zz],
                at = at,
                set.min = at[[1]], set.max = at[[length(at)]], 
                colorkey = TRUE)
  })
})
# P95
grid.arrange(grobs = p.plots[[1]], ncol = 4)
# P99
grid.arrange(grobs = p.plots[[2]], ncol = 4)

## 4. Precipitation
In this section we present the code needed to downscale precipitation. Though the steps taken are very similar to those of temperature there are some particularities that are good to mention. We start by loading the precipitation using `loadGridData` and to convert to binary the precipitation (values greater than 1mm/day of rain are set to 1 and the rest to 0).

In [ ]:
y <- loadGridData(dataset = "E-OBS_v14_0.50regular",
                  var = "pr",lonLim = c(-10,32),
                  latLim = c(36,72), 
                  years = 1979:2008)

yT <- subsetGrid(y,years = 1979:2002)
yT_bin <- binaryGrid(yT,threshold = 1,condition = "GT")
yt <- subsetGrid(y,years = 2003:2008)
yt_bin <- binaryGrid(yt,threshold = 1,condition = "GT")

We can take a look at the grid resolutions of ERA-Interim (2º) and E-OBS (0.5º) by plotting the specific humidity at 1000hPa and the observed precipitation (Figure 1 of the manuscript). 

In [ ]:
colsindex <- brewer.pal(n = 9, "BrBG")
cb <- colorRampPalette(colsindex)
coords_x <- expand.grid(xt$xyCoords$x,xt$xyCoords$y) ; names(coords_x) <- c("x","y")
coords_y <- expand.grid(yt$xyCoords$x,yt$xyCoords$y) ; names(coords_y) <- c("x","y")

pplot <- list()
pplot[[1]] <- spatialPlot(climatology(subsetGrid(xt,var = "hus@1000")), backdrop.theme = "coastline",
                          main = "Q1000 (ERA-Interim)",
                          col.regions = cb,
                          at = seq(0,0.01, 0.0001),
                          set.min = 0, set.max = 0.01, colorkey = TRUE,
                          sp.layout = list(list(SpatialPoints(coords_x), 
                                                first = FALSE, col = "black", 
                                                pch = 20, cex = 1)))
pplot[[2]] <- spatialPlot(climatology(yt), backdrop.theme = "coastline", 
                          main = "Precipitation (E-OBS)",
                          col.regions = cb,
                          at = seq(0, 4, 0.1),
                          set.min = 0, set.max = 4, colorkey = TRUE,
                          sp.layout = list(list(SpatialPoints(coords_y), 
                                                first = FALSE, col = "black", 
                                                pch = 20, cex = 1)))

lay = rbind(c(1,2))
grid.arrange(grobs = pplot, layout_matrix = lay)

We can visualize some statistics of the train and test distributions, such as the climatology, or the percentiles 2th and 98th in order to gain knowledge about the observed data (Figure 2 of the manuscript). To compute the statistics we use the library `climate4R.value`.

In [ ]:
pplot <- at <- list()
n1 <- 0; n2 <- 3
indexNames <- c("Climatology", "Frequency of rain", "P98")
for (indexName in indexNames) {
  if (indexName == "Climatology") {
    indexTrain <- valueIndex(yT,index.code = "Mean")$Index %>% redim()  
    indexTest <- valueIndex(yt,index.code = "Mean")$Index %>% redim()
    at[[1]] <- seq(0, 4, 0.1); at[[2]] <- seq(-1, 1, 0.1)
  }
  if (indexName == "Frequency of rain") {
    indexTrain <- valueIndex(yT_bin,index.code = "Mean")$Index %>% redim()  
    indexTest <- valueIndex(yt_bin,index.code = "Mean")$Index %>% redim()
    at[[1]] <- seq(0, 0.5, 0.01); at[[2]] <- seq(-0.1, 0.1, 0.01)
  }
  if (indexName == "P98") {
    indexTrain <- valueIndex(yT,index.code = "P98")$Index %>% redim()  
    indexTest <- valueIndex(yt,index.code = "P98")$Index %>% redim()
    at[[1]] <- seq(10, 20, 0.25); at[[2]] <- seq(-5, 5, 0.2)
  }
  
  for (i in 1:2) {
    if (i == 1) {
      dataset <- "(train)"; index <- indexTrain; n1 <- n1 + 1; n <- n1
      }
    if (i == 2) {
      indexTest <- gridArithmetics(indexTest,indexTrain,operator = "-")
      dataset <- "(test bias)"; index <- indexTest; n2 <- n2 + 1; n <- n2
      }
    pplot[[n]] <- spatialPlot(climatology(index), backdrop.theme = "coastline", 
                              main = paste(indexName,paste0(dataset,":"),
                                           round(mean(abs(index$Data), na.rm = TRUE),digits = 2)),
                              col.regions = cb,
                              at = at[[i]],
                              set.min = at[[i]][1], set.max = at[[i]][length(at[[i]])], 
                              colorkey = TRUE)
  }
}

lay = rbind(c(1,2,3),
            c(4,5,6))
grid.arrange(grobs = pplot, layout_matrix = lay)

### 4.1 Downscaling with generalized linear models (precipitation)

In the case of precipitation, there are 2 generalized linear models (GLMs): one to predict the occurrence of precipitation with binomial family and link logit and another to predict tha rainfall amount based on a gamma family and link logarithmic. We train the model and predict on the test set using the package `downscaleR`. In particular we train two configurations using as predictor data the closest neighbour (glm1) or the 4 closest neigbours (glm4). We obtain stochastic predictions by setting the parameter `simulate` to `TRUE`. We refer the reader to the [downscaleR wiki](https://github.com/SantanderMetGroup/downscaleR/wiki) for a detailed explanation about the working of the downscaleR functions

In [ ]:
simulateName <- c("deterministic","stochastic")
glmName <- c("glm1","glm4")
neighs <- c(1,4)
y.ocu <- binaryGrid(yT,condition = "GT",threshold = 1)
y.rest <- gridArithmetics(yT,1,operator = "-")
simulateGLM <- c(FALSE,TRUE)
lapply(1:length(glmName), FUN = function(z){
  lapply(1:length(simulateGLM),FUN = function(zz) {
    pred <- downscaleChunk(x = xT, y = y.ocu, newdata = list(xt),
                           method = "GLM", 
                           family = binomial(link = "logit"), 
                           simulate = simulateGLM[zz],
                           prepareData.args = list(local.predictors = list(n=neighs[z],
                                                                           vars = getVarNames(xT)))
    )
    pred_ocu_train <- pred[[1]] %>% redim(drop = TRUE)
    pred_ocu <- pred[[2]] %>% redim(drop = TRUE)
    rm(pred)
    pred_amo <- downscaleChunk(x = xT, y = y.rest, newdata = list(xt),
                               method = "GLM", 
                               family = Gamma(link = "log"), 
                               simulate = simulateGLM[zz],
                               condition = "GT", threshold = 0,
                               prepareData.args = list(local.predictors = list(n=neighs[z], 
                                                                               vars = getVarNames(xT))))[[2]] %>% 
      gridArithmetics(1,operator = "+") %>% redim(drop = TRUE)
    pred_bin <- binaryGrid(pred_ocu,ref.obs = yT_bin,ref.pred = pred_ocu_train); rm(pred_ocu_train)
    save(pred_bin,pred_ocu,pred_amo,
         file = paste0("./Data/precip/predictions_",simulateName[zz],"_",glmName[z],".rda"))
  })
})

### 4.2 Downscaling with deep neural networks (precipitation)
As done with temperature we define a function containing the topologies intercompared in the study. The main difference with respect to the downscaling of temperature is that for precipitation we infer the probability of rain and the shape and scale parameters of a *Gamma* distribution, and therefore there are three output parameters per predictand's gridpoint.

In [ ]:
deepName <- c("CNN-LM","CNN1","CNN10","CNN-PR","CNNdense")
architectures <- function(architecture,input_shape,output_shape) {
  if (architecture == "CNN-LM") {
    inputs <- layer_input(shape = input_shape)
    x = inputs
    l1 = layer_conv_2d(x ,filters = 50, kernel_size = c(3,3), activation = 'linear', padding = "same")
    l2 = layer_conv_2d(l1,filters = 25, kernel_size = c(3,3), activation = 'linear', padding = "same")
    l3 = layer_conv_2d(l2,filters = 1, kernel_size = c(3,3), activation = 'linear', padding = "same")
    l4 = layer_flatten(l3)
    parameter1 = layer_dense(l4,units = output_shape, activation = "sigmoid")
    parameter2 = layer_dense(l4,units = output_shape)
    parameter3 = layer_dense(l4,units = output_shape)
    outputs = layer_concatenate(list(parameter1,parameter2,parameter3))
    model <- keras_model(inputs = inputs, outputs = outputs)
  }
  
  if (architecture == "CNN1") {
    inputs <- layer_input(shape = input_shape)
    x = inputs
    l1 = layer_conv_2d(x ,filters = 50, kernel_size = c(3,3), activation = 'relu', padding = "same")
    l2 = layer_conv_2d(l1,filters = 25, kernel_size = c(3,3), activation = 'relu', padding = "same")
    l3 = layer_conv_2d(l2,filters = 1, kernel_size = c(3,3), activation = 'relu', padding = "same")
    l4 = layer_flatten(l3)
    parameter1 = layer_dense(l4,units = output_shape, activation = "sigmoid")
    parameter2 = layer_dense(l4,units = output_shape)
    parameter3 = layer_dense(l4,units = output_shape)
    outputs = layer_concatenate(list(parameter1,parameter2,parameter3))
    model <- keras_model(inputs = inputs, outputs = outputs)
  }
  
  if (architecture == "CNN10") {
    inputs <- layer_input(shape = input_shape)
    x = inputs
    l1 = layer_conv_2d(x ,filters = 50, kernel_size = c(3,3), activation = 'relu', padding = "valid")
    l2 = layer_conv_2d(l1,filters = 25, kernel_size = c(3,3), activation = 'relu', padding = "valid")
    l3 = layer_conv_2d(l2,filters = 10, kernel_size = c(3,3), activation = 'relu', padding = "valid")
    l4 = layer_flatten(l3)
    parameter1 = layer_dense(l4,units = output_shape, activation = "sigmoid")
    parameter2 = layer_dense(l4,units = output_shape)
    parameter3 = layer_dense(l4,units = output_shape)
    outputs = layer_concatenate(list(parameter1,parameter2,parameter3))
    model <- keras_model(inputs = inputs, outputs = outputs)
  }
  
  if (architecture == "CNN-PR") {
    inputs <- layer_input(shape = input_shape)
    x = inputs
    l1 = layer_conv_2d(x ,filters = 10, kernel_size = c(3,3), activation = 'relu', padding = "valid")
    l2 = layer_conv_2d(l1,filters = 25, kernel_size = c(3,3), activation = 'relu', padding = "valid")
    l3 = layer_conv_2d(l2,filters = 50, kernel_size = c(3,3), activation = 'relu', padding = "valid")
    l4 = layer_flatten(l3)
    parameter1 = layer_dense(l4,units = output_shape, activation = "sigmoid")
    parameter2 = layer_dense(l4,units = output_shape)
    parameter3 = layer_dense(l4,units = output_shape)
    outputs = layer_concatenate(list(parameter1,parameter2,parameter3))
    model <- keras_model(inputs = inputs, outputs = outputs)
  }
  
  if (architecture == "CNNdense") {
    inputs <- layer_input(shape = input_shape)
    x = inputs
    l1 = layer_conv_2d(x ,filters = 50, kernel_size = c(3,3), activation = 'relu', padding = "valid")
    l2 = layer_conv_2d(l1,filters = 25, kernel_size = c(3,3), activation = 'relu', padding = "valid")
    l3 = layer_conv_2d(l2,filters = 10, kernel_size = c(3,3), activation = 'relu', padding = "valid")
    l4 = layer_flatten(l3)
    l5 = layer_dense(l4,units = 50, activation = "relu")
    l6 = layer_dense(l5,units = 50, activation = "relu")
    parameter1 = layer_dense(l6,units = output_shape, activation = "sigmoid")
    parameter2 = layer_dense(l6,units = output_shape)
    parameter3 = layer_dense(l6,units = output_shape)
    outputs = layer_concatenate(list(parameter1,parameter2,parameter3))
    model <- keras_model(inputs = inputs, outputs = outputs)
  }
  
  return(model)
}

We prepare the predictor and predictand datasets. We substract 1 to the precipitation (*yT*) to center the conditional Gamma distribution in 0 and convert to 0 the negative values with binaryGrid in order to avoid issues with logarithms when minimizing the loss function. This will be later added to the prediction output.

In [ ]:
xy.T <- prepareData.keras(xT,binaryGrid(gridArithmetics(yT,1,operator = "-"),
                                        condition = "GE",
                                        threshold = 0,
                                        partial = TRUE),
                          first.connection = "conv",
                          last.connection = "dense",
                          channels = "last")
xy.tT <- prepareNewData.keras(xT,xy.T)
xy.t <- prepareNewData.keras(xt,xy.T)

We loop over the topologies to train the deep models and predict over the test set. Unlike GLMs where there was two models, one for the occurrence of rain and other for the amount of rain, with deep learning we minimize the negative log-likelihood of a *Bernouilli Gamma* distribution and therefore both the occurrence and quantity of rain for a given day are derived from these infered parameters. The custom loss function `bernouilliGamma.loss_function` is part of `downscaleR.keras`. In addition we use the function `bernouilliGamma.statistics` (also from `downscaleR.keras`) to compute the deterministic (i.e., expectance of the conditional distribution) or the stochastic (i.e., sample from the conditional distribution) prediction. 

In [ ]:
simulateName <- c("deterministic","stochastic")
simulateDeep <- c(FALSE,TRUE)
lapply(1:length(deepName), FUN = function(z){
  model <- architectures(architecture = deepName[z],
                         input_shape = dim(xy.T$x.global)[-1],
                         output_shape = dim(xy.T$y$Data)[2])
  downscaleTrain.keras(obj = xy.T,
             model = model,
             clear.session = TRUE,
             compile.args = list("loss" = bernouilliGamma.loss_function(last.connection = "dense"),
                                           "optimizer" = optimizer_adam(lr = 0.0001)),
                       fit.args = list("batch_size" = 100,
                            "epochs" = 1000,
                            "validation_split" = 0.1,
                            "verbose" = 1,
                            "callbacks" = list(callback_early_stopping(patience = 30),
                                callback_model_checkpoint(filepath=paste0('./models/precip/',deepName[z],'.h5'),
                                monitor='val_loss', save_best_only=TRUE))))
  lapply(1:length(simulateDeep),FUN = function(zz) {
    pred_ocu_train <- downscalePredict.keras(newdata = xy.tT,
                                             model = list("filepath" = 
                                                      paste0("./models/precip/",deepName[z],".h5"), 
                                                     "custom_objects" = 
                                                     c("custom_loss" = 
                                                      bernouilliGamma.loss_function(
                                                        last.connection = "dense"))),
                                             C4R.template = yT,
                                             clear.session = TRUE) %>% 
      subsetGrid(pred,var = "pr1")
    pred <- downscalePredict.keras(newdata = xy.t,
                                   model = list("filepath" = 
                                              paste0("./models/precip/",deepName[z],".h5"), 
                                              "custom_objects" = 
                                              c("custom_loss" = 
                                              bernouilliGamma.loss_function(last.connection = "dense"))),
                                   C4R.template = yT,
                                   clear.session = TRUE) 
    pred <- bernouilliGamma.statistics(p = subsetGrid(pred,var = "pr1"),
                                       alpha = subsetGrid(pred,var = "pr2"),
                                       beta = subsetGrid(pred,var = "pr3"),
                                       simulate = simulateDeep[zz],
                                       bias = 1)
    pred_ocu <- subsetGrid(pred,var = "probOfRain") %>% redim(drop = TRUE)
    pred_amo <- subsetGrid(pred,var = "amountOfRain") %>% redim(drop = TRUE)
    pred_bin <- binaryGrid(pred_ocu,ref.obs = yT_bin,ref.pred = pred_ocu_train); rm(pred_ocu_train)
    save(pred_bin,pred_ocu,pred_amo,file = 
            paste0("./Data/precip/predictions_",simulateName[zz],"_",deepName[z],".rda"))
  })
})

### 4.3 Validation of results (precipitation)
In this code, we calculate the validation indices by using the library `climate4R.value` of `C4R`. In particular the indices used are: the Root Mean Squared Error (RMSE), the deseasonal perason correlation, the biases of the climatology and of the percentile 2th and 98th and the ratio of the standard deviations.

In [ ]:
simulateName <- c(rep("deterministic",5),"stochastic",rep("deterministic",3))
models <- c("glm1","glm4",
               "CNN-LM","CNN1","CNN10",
               "CNN-PR","CNNdense")
measures <- c("ts.rocss","ts.RMSE","ts.rs",rep("biasRel",3),rep("bias",3))
index <- c(rep(NA,3),"Mean",rep("P98",2),"AnnualCycleRelAmp",
           "WetAnnualMaxSpell","DryAnnualMaxSpell")
validation.list <- lapply(1:length(measures), FUN = function(z) {
  lapply(1:length(models), FUN = function(zz){
    args <- list()
    load(paste0("./Data/precip/predictions_",simulateName[z],"_",models[zz],".rda"))
    if (simulateName[z] == "deterministic") {
      pred <- gridArithmetics(pred_bin,pred_amo,operator = "*")
      if (measures[z] == "ts.rocss") {
        args[["y"]] <- yt_bin; args[["x"]] <- pred_ocu
      } else if (measures[z] == "ts.RMSE") {
        args[["y"]] <- yt; args[["x"]] <- pred_amo
        args[["condition"]] = "GT"; args[["threshold"]] = 1; args[["which.wetdays"]] = "Observation"  
      } else {
        args[["y"]] <- yt; args[["x"]] <- pred
      }
    } else {
      pred <- gridArithmetics(pred_ocu,pred_amo,operator = "*")
      args[["y"]] <- yt; args[["x"]] <- pred
    }
    args[["measure.code"]] <- measures[z]
    if (!is.na(index[z])) args[["index.code"]] <- index[z]
    do.call("valueMeasure",args)$Measure
  }) %>% makeMultiGrid()
})
save(validation.list, file = "./Data/precip/validation.rda")

Once the validation indices are calculated, we represent the results in boxplots (Figure 6 of the manuscript).

In [ ]:
par(mfrow = c(3,3)) 
ylabs <- c("ROCSS","RMSE (wet days, mm)",
           "Spearman Corr.","biasRel(%)",
           "biasRel P98 (DET, %)","biasRel P98 (STO, %)",
           "Annual Cycle Rel. Amplitude", "biasRel WetAMS (days)",
           "biasRel DryAMS (days)")
lapply(1:length(validation.list), FUN = function(z) {
  if (z == 1) {ylim <- c(0.65,0.9)}
  if (z == 2) {ylim <- c(3,6.5)}
  if (z == 3) {ylim <- c(0.5,0.8)}
  if (z == 4) {ylim <- c(-0.2,0.2)}
  if (z == 5) {ylim <- c(-0.4,0.0)}
  if (z == 6) {ylim <- c(-0.2,0.2)}
  if (z == 7) {ylim <- c(-1,1)}
  if (any(z == c(8,9))) {ylim <- c(-1,1)}
  index <- (validation.list[[z]] %>% redim(drop = TRUE))$Data
  dim(index) <- c(nrow(index),prod(dim(index)[2:3]))
  indLand <- (!apply(index,MARGIN = 2,anyNA)) %>% which()
  index <- index[,indLand] %>% t()
  mglm4 <- median(index[,2],na.rm = TRUE)
  perc <- apply(index,MARGIN = 2,FUN = function(z) quantile(z,probs = c(0.1,0.9)))
  boxplot(index, outline = FALSE, ylim = ylim, range = 0.0001, ylab = ylabs[z], asp = 1)
  lines(c(0,8),c(mglm4,mglm4), col = "red")
  for (i in 1:ncol(index)) lines(c(i,i),perc[,i], lty = 2)
})

In order to obtain a spatial representation of the validation indices computed above we use the function `spatialPlot` of `visualizeR`. In particular, we plot the ROCSS, the spearman correlation and the relative biases of the mean and the P98 for the glm1, glm4 and CNN1 models (Figure 7 of the manuscript).

In [ ]:
ylabs <- c("glm1","glm4",NA,"CNN1")
mains <- c("ROCSS",NA,"Spearman Corr.","biasRel","biasRel P98")
cb <- colorRampPalette(brewer.pal(9, "OrRd"))(80)
colsindex <- rev(brewer.pal(n = 9, "RdBu"))
cb2 <- colorRampPalette(colsindex)
validation.plots <- lapply(c(1,3,4,5),FUN = function(z) {
  lapply(c(1,2,4),FUN = function(zz) {
    if (z == 1) {
      at <- seq(0.5, 1, 0.01); colorbar <- cb
    } else if (z == 3) {
      at <- seq(0.5, 1, 0.02); colorbar <- cb
    } else {
      at <- seq(-0.5, 0.5, 0.01); colorbar <- cb2
    }
    index <- subsetDimension(validation.list[[z]],dimension = "var",indices = zz) %>% redim(drop = TRUE)
    spatialPlot(index, backdrop.theme = "coastline",
                ylab = ylabs[zz],
                main = paste(mains[z],
                             round(mean(abs(index$Data), na.rm = TRUE), digits = 2)),
                col.regions = colorbar,
                at = at,
                set.min = at[1], set.max = at[length(at)], colorkey = TRUE)
  })
})
lay = cbind(1:3,4:6,7:9,10:12)
grid.arrange(grobs = unlist(validation.plots,recursive = FALSE), layout_matrix = lay)

## 5. Technical aspects
To perform all the stages involved in this study we relied on the local machine described below.
  
1. Local Machine (HP-ProDesk-600-G2-MT)
     + Operating system: ubuntu 4.15.0-72-generic
     + Memory: 15.6 GiB
     + Processor: Intel® Core™ i7-6700 CPU @ 3.40GHz × 8 
     + SO: 64 bits
     + Disc: 235.1 GiB